In [563]:
#Importing dependancies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import psycopg2 as ps
from psycopg2 import sql, connect
from getpass import getpass

In [564]:
#Importing Sklearn
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [565]:
#Connecting to AWS
import getpass
engine = ps.connect(
    database="postgres",
    user= input("Username: "),
    password= getpass.getpass("Password: "),
    host="highered-db.cx2ll8zuuepz.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [566]:
cr = engine.cursor()
cr.execute('SELECT * FROM highered_data_table;')
tmp = cr.fetchall()

In [567]:
#Converting to Pandas DataFrame
df = pd.DataFrame(tmp, columns= ['Year','UNITID','FACSTAT','ARANK','HRTOTLT','HRTOTLM','HRTOTLW','INSTNM','CITY','STABBR','ZIP',
                                    'CONTROL','HLOFFER','INSTSIZE','LONGITUD','LATITUDE'])
df

,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192599,2020,232618,0,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192600,2020,232618,10,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192601,2020,232618,10,6,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192602,2020,232618,40,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100


In [568]:
# #Loading the cleaned Data
# #file_path = Path('highered_merged_data.csv')
# #df = pd.read_csv(file_path)
# df.head(5)



In [569]:
# df_test = df.drop_duplicates()
df_test = df.drop(['FACSTAT', 'ARANK'], axis=1)
df_test.sample(10)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
72325,2021,172264,2,1,1,Siena Heights University,Adrian,MI,49221,2,7,2,-84.018841,41.905624
143915,2021,447582,51,27,24,New River Community and Technical College,Beaver,WV,25813,1,3,2,-81.136085,37.767670
91388,2019,217633,159,72,87,Anderson University,Anderson,SC,29621-4035,2,9,2,-82.640358,34.515200
145936,2021,458210,30,12,18,West Coast University-Orange County,Anaheim,CA,92802,3,9,2,-117.911811,33.811040
24577,2021,100751,2,2,0,The University of Alabama,Tuscaloosa,AL,35487,1,9,5,-87.545978,33.211875
107590,2021,211307,4,1,3,Bucks County Community College,Newtown,PA,18940,1,3,3,-74.965841,40.239102
189530,2020,226116,1,1,0,Lamar State College-Port Arthur,Port Arthur,TX,77640,1,3,2,-93.925886,29.880385
161296,2020,163046,22,10,12,Loyola University Maryland,Baltimore,MD,21210,2,9,3,-76.621872,39.346358
27503,2019,147767,2296,1366,930,Northwestern University,Evanston,IL,60208,2,9,5,-87.679858,42.050356
66087,2021,164465,38,19,19,Amherst College,Amherst,MA,1002,2,5,2,-72.518493,42.372459


In [570]:
#Dropping duplicates
df_test.drop_duplicates(inplace=True)

In [571]:
# df_test1 = df_test.groupby(['UNITID'], as_index=False).agg({'Year': 'first','HRTOTLT': 'sum', 'HRTOTLM': 'sum', 'HRTOTLW': 'sum'})
# df_test1.sample(10)

In [572]:
df_test1 = df_test.groupby(['UNITID', 'Year'])['HRTOTLT'].sum().unstack(fill_value=0)
#.unstack(fill_value=0)
df_test1.head(10)

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
100751,6746,6708,6928
100760,137,127,119
100812,274,256,292
100830,865,1011,1004


In [573]:
df_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 100654 to 497286
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2019    3756 non-null   int64
 1   2020    3756 non-null   int64
 2   2021    3756 non-null   int64
dtypes: int64(3)
memory usage: 117.4 KB


In [574]:
df_test1.dropna()

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
...,...,...,...
496681,0,0,10
496991,0,0,7
497046,0,0,112


In [575]:
#Using drop() to delete rows based on column value
df_test1.drop(df_test1[df_test1[2019] == 0].index, inplace = True)

In [576]:
#Calculating retention rate
df_test1["Retention_Rate"] = (df_test1[2021]/df_test1[2019] * 100).round(2).astype(float)
df_test1.sample(10)

Year,2019,2020,2021,Retention_Rate
UNITID,,,,
148876,30,22,18,60.00
487597,37,25,36,97.30
439817,13,0,0,0.00
166629,6243,6168,6234,99.86
108092,1168,1200,1180,101.03
363907,40,34,34,85.00
171395,97,60,88,90.72
144050,12891,12985,14271,110.71
222576,771,747,650,84.31


In [577]:
#Adding retention boolean
df_test1["Retention"] = np.where(df_test1['Retention_Rate'] > 99.99, True, False)
df_test1

Year,2019,2020,2021,Retention_Rate,Retention
UNITID,,,,,
100654,874,1176,1027,117.51,True
100663,9874,9961,10208,103.38,True
100690,117,100,120,102.56,True
100706,1578,1514,1534,97.21,False
100724,1026,996,970,94.54,False
...,...,...,...,...,...
494287,7,6,8,114.29,True
494436,13,13,15,115.38,True
494737,19,19,19,100.00,True


In [578]:
#Dropping Retention Rate Column
df_test1.drop(columns=["Retention"])

Year,2019,2020,2021,Retention_Rate
UNITID,,,,
100654,874,1176,1027,117.51
100663,9874,9961,10208,103.38
100690,117,100,120,102.56
100706,1578,1514,1534,97.21
100724,1026,996,970,94.54
...,...,...,...,...
494287,7,6,8,114.29
494436,13,13,15,115.38
494737,19,19,19,100.00


In [579]:
#Creating feature
X = df_test1.drop("Retention", axis=1)
X = pd.get_dummies(X)

In [580]:
#Adding a target thought? 
target = ["Retention"]

In [581]:
#Creating a target
y = df_test1.loc[:, target].copy()

In [582]:
#Decribing
X.describe()

Year,2019,2020,2021,Retention_Rate
count,3645.000000,3645.000000,3645.000000,3645.000000
mean,799.189300,776.890809,777.718519,101.006209
std,2164.002758,2098.881805,2145.416521,216.722722
min,1.000000,0.000000,0.000000,0.000000
25%,50.000000,46.000000,44.000000,86.570000
50%,212.000000,201.000000,200.000000,96.740000
75%,601.000000,584.000000,585.000000,105.740000
max,36293.000000,30347.000000,35552.000000,12900.000000


In [583]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train['Retention']))
print(Counter(y_test['Retention']))

Counter({False: 1614, True: 1119})
Counter({False: 526, True: 386})


In [584]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Counter({False: 1614, True: 1119})


In [585]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


1.0

In [586]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[526,   0],
       [  0, 386]], dtype=int64)

In [587]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       1.00      1.00      1.00      1.00      1.00      1.00       526
       True       1.00      1.00      1.00      1.00      1.00      1.00       386

avg / total       1.00      1.00      1.00      1.00      1.00      1.00       912



In [588]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Counter({False: 1614, True: 1119})


In [589]:
# Calculated the balanced accuracy score
y_pred = ee_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


1.0

In [590]:
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_pred)
cm2

array([[526,   0],
       [  0, 386]], dtype=int64)